In [ ]:
!pip install netket

In [ ]:
import netket as nk
import numpy as np

def lanczos_gs(n_sites, pbc=True):

    graph = nk.graph.Hypercube(length=n_sites, n_dim=1, pbc=pbc)
    hi = nk.hilbert.Spin(s=0.5, total_sz=0, N=graph.n_nodes)
    hamiltonian = nk.operator.Heisenberg(hilbert=hi, graph=graph, J=1.0)

    result = nk.exact.lanczos_ed(hamiltonian, k=1, compute_eigenvectors=False)

    return result[0]


N = 12
exact_energy = lanczos_gs(N)

print(f"Lattice Size: {N}")
print(f"Exact Ground State Energy: {exact_energy:.6f}")
print(f"Energy per site: {exact_energy/N:.6f}")

In [ ]:
import netket as nk
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

def get_comparison_data(n_sites_list):
    results = []

    for n in n_sites_list:
        print(f"--- Analyzing N = {n} ---")

        # Lattice and Hamiltonian
        #graph = nk.graph.Hypercube(length=n, n_dim=1, pbc=True)
        graph=nk.graph.Chain(n)
        hi = nk.hilbert.Spin(s=0.5, total_sz=0, N=graph.n_nodes)
        hamiltonian = nk.operator.Heisenberg(hilbert=hi, graph=graph)

        # Exact Ground State (Lanczos)
        e_exact = None
        if n <= 22:
            ed_result = nk.exact.lanczos_ed(hamiltonian, k=1)
            e_exact = float(ed_result[0])

        # 3. Neural Network (RBM)
        model = nk.models.RBM(alpha=2, param_dtype=float)
        sampler = nk.sampler.MetropolisExchange(hi, graph=graph)

        vstate = nk.vqs.MCState(sampler, model, n_samples=1008)

# Neel state

        n_chains = vstate.sampler.n_chains
        neel = np.ones(n)
        neel[1::2] = -1

        neel_samples = np.tile(neel, (n_chains, 1)).astype(np.int8)


        vstate.sampler_state = vstate.sampler_state.replace(σ=jnp.array(neel_samples))

        # Optimizer: Stochastic Reconfiguration (SR)
        optimizer = nk.optimizer.Sgd(learning_rate=0.02)
        sr = nk.optimizer.SR(diag_shift=0.1)
        vmc = nk.driver.VMC(hamiltonian, optimizer, variational_state=vstate, preconditioner=sr)

        #log = nk.logging.RuntimeLog()

        # Optimization
        vmc.run(n_iter=400,out=log)
        history = log.data
        print(history['Energy'].Mean)


        stats = vstate.expect(hamiltonian)
        e_vmc = float(stats.mean.real)
        err_vmc = float(stats.error_of_mean)

        results.append({
            "N": n,
            "Exact": e_exact,
            "VMC": e_vmc,
            "VMC_err": err_vmc,
            "Error": abs(e_exact - e_vmc) if e_exact else None
        })

    return results;
#n_sizes = [10,16,20,22,24,26]
n_sizes = [10,16,24]
data = get_comparison_data(n_sizes)

# 4. Print

for d in data:
    status = f"Diff: {d['Error']:.6f}" if d['Error'] is not None else "Exact not computed"
    print(f"N={d['N']} | VMC: {d['VMC']:.5f} | {status}")

In [ ]:
import matplotlib.pyplot as plt

# 1. Extract data for plotting
# Absolute error plot (only where Exact exists)
n_vals_error = [d['N'] for d in data if d['Error'] is not None]
error_vals = [d['Error'] for d in data if d['Error'] is not None]

# Data for Energy per Site (all N)
n_vals_all = [d['N'] for d in data]
vmc_en_per_site = [d['VMC'] / d['N'] for d in data]
vmc_err_per_site = [d['VMC_err'] / d['N'] for d in data] # Error normalized by N

# Exact values for comparison (only where they exist)
n_vals_exact = [d['N'] for d in data if d['Exact'] is not None]
exact_en_per_site = [d['Exact'] / d['N'] for d in data if d['Exact'] is not None]

# 2. Create the Figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

#Plot 1: Absolute Error (Log Scale)
if error_vals:
    ax1.plot(n_vals_error, error_vals, 'o--', color='red', label='VMC Absolute Error')
    ax1.set_yscale('log')
    ax1.set_xlabel('System Size (N)')
    ax1.set_ylabel(r'Abs Error $|E_{exact} - E_{VMC}|$')
    ax1.set_title('Convergence Error vs System Size')
    ax1.grid(True, which="both", ls="-", alpha=0.2)
    ax1.legend()
else:
    ax1.text(0.5, 0.5, 'No Exact Data available for Error Plot',
             ha='center', va='center', transform=ax1.transAxes)

# Plot 2: Energy per Site with Statistical Error Bars 
# We use errorbar() to show the Monte Carlo uncertainty
ax2.errorbar(n_vals_all, vmc_en_per_site, yerr=vmc_err_per_site,
             fmt='s-', color='tab:blue', ecolor='black', capsize=4,
             label='VMC $E/N$', alpha=0.8)

# Overlay Exact values if they exist
if exact_en_per_site:
    ax2.plot(n_vals_exact, exact_en_per_site, 'x',
             color='black', markersize=8, label='Exact $E/N$')

ax2.set_xlabel('System Size (N)')
ax2.set_ylabel('Energy per Site ($E/N$)')
ax2.set_title('Ground State Energy Density')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
#plt.savefig("heisenberg_benchmark_with_errors.png")
plt.show()

In [ ]:
import netket as nk
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

def get_comparison_data(n_sites_list):
    results = []

    for n in n_sites_list:
        print(f"--- Analyzing N = {n} ---")

        # Lattice and Hamiltonian
        #graph = nk.graph.Hypercube(length=n, n_dim=1, pbc=True)
        graph=nk.graph.Chain(n)
        hi = nk.hilbert.Spin(s=0.5, total_sz=0, N=graph.n_nodes)
        hamiltonian = nk.operator.Heisenberg(hilbert=hi, graph=graph)



        # 3. Neural Network (RBM)
        model = nk.models.RBM(alpha=1, param_dtype=float)
        sampler = nk.sampler.MetropolisExchange(hi, graph=graph)

        vstate = nk.vqs.MCState(sampler, model, n_samples=1008)

# Neel state

        n_chains = vstate.sampler.n_chains
        neel = np.ones(n)
        neel[1::2] = -1

        neel_samples = np.tile(neel, (n_chains, 1)).astype(np.int8)


        vstate.sampler_state = vstate.sampler_state.replace(σ=jnp.array(neel_samples))

        # Optimizer: Stochastic Reconfiguration (SR)
        optimizer = nk.optimizer.Sgd(learning_rate=0.02)
        sr = nk.optimizer.SR(diag_shift=0.1)
        #vmc = nk.driver.VMC(hamiltonian, optimizer, variational_state=vstate, preconditioner=sr)
        vmc = nk.driver.VMC_SR(hamiltonian, optimizer, diag_shift=0.01, variational_state=vstate)
        # Run Optimization
        vmc.run(n_iter=200)
        e_vmc = float(vstate.expect(hamiltonian).mean.real)

        results.append({
            "N": n,
            "VMC": e_vmc,
        })

    return results


n_sizes = [144]
data = get_comparison_data(n_sizes)

# 4. Print

for d in data:
    #status = f"Diff: {d['Error']:.6f}" if d['Error'] is not None else "Exact not computed"
    print(f"N={d['N']} | VMC: {d['VMC']:.5f} ")